In [1]:
import re
import os
import ollama
import numpy as np
import pandas as pd
import json

In [2]:
master_file_path = "/home/wrishav/Desktop/Workspace/SKU-Retail/redundant/data_master.csv"
m_columns = ['itemcode', 'catcode', 'category', 'subcat', 'ssubcat', 'company', 'mbrand', 'brand', 'sku',
             'packtype', 'base_pack', 'flavor', 'color', 'wght', 'uom', 'mrp']
master_file = pd.read_csv(master_file_path, usecols=m_columns)
master_dict = master_file.to_dict(orient='index')
master_file.head()

,itemcode,catcode,category,subcat,ssubcat,company,mbrand,brand,sku,packtype,base_pack,flavor,color,wght,uom,mrp
0,99765328,105,CIGARETTE,NaN,NaN,GORKHA LAHARI PVT.LTD.- NEPAL,CAPTAIN,CAPTAIN GOLD FILTER KINGS,HL 20'S,HL,20,FULL FLAVOUR,NaN,20.0,NO,161.0
1,99765334,105,CIGARETTE,NaN,NaN,JT INTERNATIONAL - SA,CAMEL CONNECT,CAMEL CONNECT ORIGINAL,HL 20'S,HL,20,FULL FLAVOUR,NaN,20.0,NO,161.5
2,99765335,105,CIGARETTE,NaN,NaN,JT INTERNATIONAL - SA,CAMEL CONNECT,CAMEL CONNECT MINT CRUSH,HL 20'S,HL,20,FRESH,NaN,20.0,NO,239.0
3,99765387,105,CIGARETTE,NaN,NaN,GORKHA LAHARI PVT.LTD.- NEPAL,NEPAL GOLD FLAKE,NEPAL GOLD FLAKE PREMIUM FILTER,HL 20'S,HL,20,FULL FLAVOUR,NaN,20.0,NO,133.5
4,90110252,105,CIGARETTE,NaN,NaN,JT INTERNATIONAL - SA,WINSTON,WINSTON CLASSIC,HL 20'S,HL,20,FULL FLAVOUR,NaN,20.0,NO,180.0


In [3]:
transaction_file_path = "/home/wrishav/Desktop/Workspace/SKU-Retail/redundant/data_transaction.csv"
t_columns = ['CATEGORY', 'MANUFACTURE', 'BRAND', 'ITEMDESC', 'MRP', 'PACKSIZE', 'PACKTYPE']
transaction_file = pd.read_csv(transaction_file_path, usecols=t_columns)
transaction_dict = transaction_file.to_dict(orient='index')
transaction_file.head()

,CATEGORY,MANUFACTURE,BRAND,ITEMDESC,MRP,PACKSIZE,PACKTYPE
0,105.0,SURYA NEPAL PVT LTD - NEPAL,SURYA FUSION,SURYA FUSION/HL/20NOS/MRP320,320.0,20 NOS,KS


In [4]:
def format_master(master_row):
    return f"""
Item Code: {master_row['itemcode']}
Category Code: {master_row['catcode']}
Category: {master_row['category']}
Subcategory: {master_row['subcat']}
Sub-Subcategory: {master_row['ssubcat']}
Company: {master_row['company']}
Main Brand: {master_row['mbrand']}
Brand: {master_row['brand']}
Pack Type: {master_row['packtype']}
Pack Size: {master_row['base_pack']}
Flavor: {master_row['flavor']}
Color: {master_row['color']}
Unit of Measure: {master_row['uom']}
MRP: {master_row['mrp']}
    """

In [5]:
def format_transaction(transaction_row):
    return f"""
Category Code: {transaction_row['CATEGORY']}
Company: {transaction_row['MANUFACTURE']}
Brand: {transaction_row['BRAND']}
Item Description: {transaction_row['ITEMDESC']}
MRP: {transaction_row['MRP']}
Pack Size: {re.match(r'(\d+)\s*(.*)', transaction_row['PACKSIZE']).group(1)}
Unit of Measure: {re.match(r'(\d+)\s*(.*)', transaction_row['PACKSIZE']).group(2)}
Pack Type: {transaction_row['PACKTYPE']}
    """

In [6]:
def format_context_matches(master_dictionary) -> str:
    """Format context matches for the LLM prompt"""
    context_lines = []
    
    for idx, master_row in master_dictionary.items():
        formatted_row = format_master(master_row)
        context_lines.append(f"Context item {idx}:{formatted_row}")
    return "\n".join(context_lines)

In [7]:
def format_query_match(transaction_dictionary) -> str:
    """Format context matches for the LLM prompt"""
    context_lines = []
    
    for idx, master_row in transaction_dictionary.items():
        formatted_row = format_transaction(master_row)
        context_lines.append(f"Transaction item {idx}:{formatted_row}")
    return "\n".join(context_lines)

In [8]:
def call_llm(prompt):
    response = ollama.generate(
    model="gemma3:4b",
    prompt=prompt,
    stream=False,
    format="json",
    options={
        "temperature": 0.1,      # Low temperature for consistency
        "top_p": 0.9,
        "num_predict": 50,       # Short response expected
        "stop": ["\n\n", "EXPLANATION", "REASONING"]  # Stop early
        }
    )
    response_text = response['response'].strip()
    return response_text

In [9]:
def generate_prompt(context_rows, transaction_row):
  prompt = f"""
You are an expert product matching AI.
You have been given a transaction item and a set of context items from the master catalog.
Determine which context item best matches the transaction item.

Here are the context items from the master catalog:
\n{format_context_matches(context_rows)}

Here is the transaction item to match:
\n{format_query_match(transaction_row)}

Matching criteria (Priority Order):
1. Category code alignment
2. EXACT company match
3. EXACT brand match
4. EXACT pack size and pack type match
5. MRP/price similarity

Instructions:
1. Compare transaction item with each context item carefully
2. Prioritize exact matches in category code, company, brand, pack size, and pack type
3. Consider MRP similarity as a secondary factor

Based on the attributes provided, identify the best matching context item for the transaction item.
Respond strictly with a JSON object in the following format:
{{
  "context_item": "<The context item number, for example, 0 if 'Context item 0'>",
  "score": "<The confidence score normalized between 0 and 1, with 1 being a perfect match>",
}}
  """
  return prompt

In [10]:
chunksize = 10
context_len = len(master_dict)
responses = []

start = 0
while start < context_len:
    end = min(start + chunksize, context_len)
    print(f"Processing {start} to {end-1}\nChunk start index: {start}\n")
    context_rows = {i: master_dict[i] for i in range(start, end)}
    transaction_row = {0: transaction_dict[0]}
    prompt = generate_prompt(context_rows, transaction_row)
    response = call_llm(prompt)
    responses.append(response)
    print(f"LLM Response:\n{response}\n")
    start += chunksize

Processing 0 to 9
Chunk start index: 0

LLM Response:
{"context_item": "3", "score": 0.9}

Processing 10 to 16
Chunk start index: 10

LLM Response:
{
  "context_item": "10",
  "score": 1.0
}



In [11]:
print("JSON Parsed Responses:")
for i, response in enumerate(responses):
    try:
        parsed = json.loads(response)
        print(f"Response {i}: {parsed}")
    except json.JSONDecodeError:
        print(f"Response {i}: Invalid JSON")

JSON Parsed Responses:
Response 0: {'context_item': '3', 'score': 0.9}
Response 1: {'context_item': '10', 'score': 1.0}


In [12]:
new_master = {}
for i, response in enumerate(responses):
    try:
        parsed = json.loads(response)
        context_index = int(parsed['context_item'])
        score = float(parsed['score'])
        if score >= 0.75:  # Threshold for a good match
            new_master[i] = master_dict[context_index]
    except (json.JSONDecodeError, KeyError, ValueError):
        continue

new_master

{0: {'itemcode': 99765387,
  'catcode': 105,
  'category': 'CIGARETTE',
  'subcat': nan,
  'ssubcat': nan,
  'company': 'GORKHA LAHARI PVT.LTD.- NEPAL',
  'mbrand': 'NEPAL GOLD FLAKE',
  'brand': 'NEPAL GOLD FLAKE PREMIUM FILTER',
  'sku': "HL 20'S",
  'packtype': 'HL',
  'base_pack': 20,
  'flavor': 'FULL FLAVOUR',
  'color': nan,
  'wght': 20.0,
  'uom': 'NO',
  'mrp': 133.5},
 1: {'itemcode': 99765321,
  'catcode': 105,
  'category': 'CIGARETTE',
  'subcat': nan,
  'ssubcat': nan,
  'company': 'SURYA NEPAL PVT LTD - NEPAL',
  'mbrand': 'SURYA',
  'brand': 'SURYA FUSION',
  'sku': "HL 20'S",
  'packtype': 'HL',
  'base_pack': 20,
  'flavor': 'FULL FLAVOUR',
  'color': nan,
  'wght': 20.0,
  'uom': 'NO',
  'mrp': 320.0}}

In [13]:
chunksize = 10
context_len = len(new_master)
new_responses = []

start = 0
while start < context_len:
    end = min(start + chunksize, context_len)
    print(f"Processing {start} to {end-1}\nChunk start index: {start}\n")
    context_rows = {i: new_master[i] for i in range(start, end)}
    transaction_row = {0: transaction_dict[0]}
    prompt = generate_prompt(context_rows, transaction_row)
    response = call_llm(prompt)
    new_responses.append(response)
    print(f"LLM Response:\n{response}\n")
    start += chunksize

Processing 0 to 1
Chunk start index: 0

LLM Response:
{
  "context_item": "1",
  "score": "0.95"
}



In [14]:
print("JSON Parsed Responses:")
for i, response in enumerate(new_responses):
    try:
        parsed = json.loads(response)
        print(f"Response {i}: {parsed}")
    except json.JSONDecodeError:
        print(f"Response {i}: Invalid JSON")

JSON Parsed Responses:
Response 0: {'context_item': '1', 'score': '0.95'}


In [15]:
def reduce_once(
    candidates: dict,
    transaction_row: dict,
    chunksize: int
) -> dict:
    """
    candidates: {absolute_id: master_row}
    returns: reduced {absolute_id: master_row}
    """

    candidate_ids = list(candidates.keys())
    reduced = {}

    start = 0
    while start < len(candidate_ids):
        end = min(start + chunksize, len(candidate_ids))
        chunk_ids = candidate_ids[start:end]

        context_rows = {
            i: candidates[cid]
            for i, cid in enumerate(chunk_ids)
        }

        prompt = generate_prompt(context_rows, transaction_row)
        response = call_llm(prompt)

        try:
            parsed = json.loads(response)
            chunk_idx = int(parsed["context_item"])
            winner_id = chunk_ids[chunk_idx]

            reduced[winner_id] = candidates[winner_id]

        except (json.JSONDecodeError, KeyError, ValueError, IndexError):
            pass

        start += chunksize

    return reduced

In [16]:
def tournament_match(
    master_dict: dict,
    transaction_row: dict,
    chunksize: int = 100,
    max_iterations: int = 20
):
    candidates = master_dict
    iteration = 1

    while len(candidates) > 1 and iteration <= max_iterations:
        print(
            f"Iteration {iteration}: "
            f"{len(candidates)} candidates → "
            f"{(len(candidates) + chunksize - 1) // chunksize} chunks"
        )

        candidates = reduce_once(
            candidates=candidates,
            transaction_row=transaction_row,
            chunksize=chunksize
        )

        iteration += 1

    return candidates

In [17]:
final_candidate = tournament_match(
    master_dict=master_dict,
    transaction_row={0: transaction_dict[0]},
    chunksize=10
)

print("Final match:", final_candidate)

Iteration 1: 17 candidates → 2 chunks
Iteration 2: 2 candidates → 1 chunks
Final match: {10: {'itemcode': 99765321, 'catcode': 105, 'category': 'CIGARETTE', 'subcat': nan, 'ssubcat': nan, 'company': 'SURYA NEPAL PVT LTD - NEPAL', 'mbrand': 'SURYA', 'brand': 'SURYA FUSION', 'sku': "HL 20'S", 'packtype': 'HL', 'base_pack': 20, 'flavor': 'FULL FLAVOUR', 'color': nan, 'wght': 20.0, 'uom': 'NO', 'mrp': 320.0}}
